"""This is a challenge, using all the ~knowledge~ I gained learning more about word embeddings and tflearn in 
the imdb review video, I will attempt this independently on an ign video game review dataset.  
"""


 Using the existing labels is extra credit. The baseline is that you can just convert the labels so that there are only 2 emotions (positive or negative). Ideally you can use an RNN via TFLearn like the one in this example, but I'll accept other types of ML models as well.

In [8]:
import pandas as pd
import numpy as np
from tflearn.data_utils import to_categorical, pad_sequences, VocabularyProcessor
import tflearn

In [2]:
data = pd.read_csv('ign.csv')

data = data[['title','score_phrase','score']]

In [3]:

#Start with binary classification. 
good_games = data.loc[data['score'] > 6.0]
bad_games = data.loc[data['score'] < 4.0]

In [4]:
print("There are ",len(bad_games)," negatively reviewed games vs. ",
      len(good_games)," positive.")

There are  1079  negatively reviewed games vs.  13324  positive.


In [5]:
data.score_phrase.value_counts()

Great          4773
Good           4741
Okay           2945
Mediocre       1959
Amazing        1804
Bad            1269
Awful           664
Painful         340
Unbearable       72
Masterpiece      55
Disaster          3
Name: score_phrase, dtype: int64

A comment. There are far more good games than bad games, so at worse picking every game as positive vs. negative would lead to 13/16 results, so some further process should be done, but for now just working with binary for a proof of concept. 

In [9]:
titles_good = good_games['title'].values
y_good = np.ones(len(titles_good))

titles_bad =bad_games['title'].values
y_bad = 0*np.ones(len(titles_bad))


X = np.concatenate((titles_good,titles_bad))
y = np.concatenate((y_good,y_bad))

y = to_categorical(y, nb_classes=2)


In [12]:
from sklearn.model_selection import train_test_split



X_train, X_test, y_train, y_test = train_test_split(X,y)

/Users/noahkasmanoff/anaconda/lib/python3.6/importlib/_bootstrap.py:205: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/Users/noahkasmanoff/anaconda/lib/python3.6/importlib/_bootstrap.py:205: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


In [13]:
#now convert X_train using vocab processor, and X_test gets transformed after. 


vocab_proc = VocabularyProcessor(30)  #
#do it with all
X =  np.array(list(vocab_proc.fit_transform(X)))
#X_train = np.array(list(vocab_proc.fit_transform(X_train)))
#X_test =  np.array(list(vocab_proc.transform(X_test)))

X_train, X_test, y_train, y_test = train_test_split(X,y)

In [14]:
import tflearn
net = tflearn.input_data([None,30]) #input layer, [batch_size, size of input]
net = tflearn.embedding(net, input_dim=10000, output_dim=128)  #we use the previous layers output as the next layers input. 
# in an embedding, words are represented by dense vectors where 
#a vector represents the projection of the word into a continuous
#vector space. There are 10k words in these reviews, so that is how many dim are needed. 

#The position of a word in the learned vector space is referred to as its embedding.

# a word embedding can be learned as part of a deep learning model. 
#This can be a slower approach, but tailors the model to a specific
#training dataset.
net = tflearn.lstm(net, 128, dropout=0.8)
net = tflearn.fully_connected(net, 2, activation='softmax') #connects the layers. not just adjascent neurons. 
net = tflearn.regression(net, optimizer='adam', learning_rate=0.001,#applies a regression to the input. 
                         loss='categorical_crossentropy')



/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signatur

/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signatur

Instructions for updating:
keep_dims is deprecated, use keepdims instead


/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))


In [15]:
# Training
model = tflearn.DNN(net, tensorboard_verbose=0)
model.fit(X_train, y_train, validation_set=(X_test, y_test), show_metric=True,
          batch_size=32)

###Now my stuff!
#Testing

score = model.score(testX,testY)

print("The results of this model are ", 100*score,"% accuracy. ")

Training Step: 337  | total loss: 0.24068 | time: 20.944s
| Adam | epoch: 001 | loss: 0.24068 - acc: 0.9354 -- iter: 10784/10802


KeyboardInterrupt: 

End result: This works! However a RNN for binary classification isn't the best idea in this case, as there are far more positively reviewed games than negative ones. In the end I used a vocabulary processor to provide a sequential nature to each game, and the RNN slowly tried to pick up on words in the game's title that give away the fact that the game is good or not. This isn't a solid idea in practice, but I guess the model could pick up on games in a series, such as "call of duty". Next up, I'll be learning more about word2vec and how word embedding can be used to pick up on relationships in a corpus.